## 1. Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2. Set Project Path

**⚠️ Important:** Change the path below to match your actual project folder location in Drive

In [ ]:
import os

# Project path in Google Drive - CHANGE THIS!
PROJECT_PATH = '/content/drive/MyDrive/sr-fedopt'

# Change to project directory
os.chdir(PROJECT_PATH)
print(f"Current directory: {os.getcwd()}")
print(f"Files: {os.listdir('.')}")

## 3. Install Dependencies

In [ ]:
!pip install -q -r requirements.txt
print("✅ Dependencies installed successfully!")

## 4. Check GPU Availability

In [ ]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ WARNING: GPU not available! Enable it in Runtime → Change runtime type → GPU")

## 5. Run Experiments

### Option A: Quick Test (5 minutes)
Run a small test to verify everything works

In [ ]:
# Create a quick test configuration
import json

test_config = {
    "test": [
        {
            "dataset": ["mnist"],
            "net": ["logistic"],
            "iterations": [2],
            "n_clients": [10],
            "participation_rate": [0.5],
            "classes_per_client": [10],
            "batch_size": [1],
            "balancedness": [1.0],
            "momentum": [0.0],
            "compression": [["none", {}]],
            "log_frequency": [5],
            "optimizer": ["SGD"],
            "server_optimizer": ["fedavg"],
            "server_beta1": [0.9],
            "server_beta2": [0.999],
            "server_eps": [1e-8],
            "server_lr": [1.0],
            "shrinkage_mode": ["global"],
            "shrinkage_scope": ["all"],
            "sigma_source": ["inter_client"],
            "client_update_noise_std": [0.0],
            "base_log_path": ["results/colab_test/"],
            "seeds": [42]
        }
    ]
}

# Save temporarily
with open('federated_learning_test.json', 'w') as f:
    json.dump(test_config, f, indent=2)

print("✅ Test config created!")

In [ ]:
# Run quick test
!python federated_learning.py --schedule test

### Option B: Run All Experiments (~4-5 hours)

**⚠️ Warning:** This will run 180 experiments and may take several hours!

**Breakdown:**
- MNIST + FashionMNIST (logistic): 120 runs → ~2 hours
- CIFAR-10 (simple_cnn): 60 runs → ~2-3 hours

In [ ]:
# Run all experiments at once
!python federated_learning.py --schedule main

### Option C: Run in Batches (Recommended ⭐)

To avoid Colab timeouts, run experiments in smaller batches.
If disconnected, you can continue from where you left off.

In [ ]:
# Batch 1: MNIST experiments (experiments 0-59)
# Estimated time: ~1 hour
# !python federated_learning.py --schedule main --start 0 --end 60

In [ ]:
# Batch 2: FashionMNIST experiments (experiments 60-119)
# Estimated time: ~1 hour
# !python federated_learning.py --schedule main --start 60 --end 120

In [ ]:
# Batch 3: CIFAR-10 experiments (experiments 120-179)
# Estimated time: ~2-3 hours
# !python federated_learning.py --schedule main --start 120 --end 180

## 6. Monitor Progress

In [ ]:
# Check how many experiments have completed
!find results/main -name '*.npz' | wc -l

In [ ]:
# List result directories
!find results/main -type d | head -30

## 7. Generate Plots and Statistics

In [ ]:
# Generate all plots and CSV files
!python tools/visualize_results.py

In [ ]:
# Display plots in notebook
from IPython.display import Image, display
import os

plot_dir = 'results/plots'
if os.path.exists(plot_dir):
    plots = [f for f in os.listdir(plot_dir) if f.endswith('.png')]
    if plots:
        print(f"📊 Found {len(plots)} plots\n")
        # Display first 3 plots as examples
        for p in sorted(plots)[:3]:
            print(f"\n{'='*60}")
            print(f"📈 {p}")
            print('='*60)
            display(Image(filename=os.path.join(plot_dir, p)))
    else:
        print("No plots found yet. Run visualize_results.py first.")
else:
    print(f"Plot directory not found: {plot_dir}")

In [ ]:
# Show summary statistics from CSV files
import pandas as pd
import glob

csv_files = glob.glob('results/plots/*_statistics_summary.csv')
if csv_files:
    print("📊 Summary Statistics:\n")
    for csv_file in sorted(csv_files):
        print(f"\n{'='*60}")
        print(f"📄 {os.path.basename(csv_file)}")
        print('='*60)
        df = pd.read_csv(csv_file)
        print(df.to_string(index=False))
else:
    print("No CSV files found. Run visualize_results.py first.")

## 8. Download Results (Optional)

Compress and download all results to your local machine

In [ ]:
# Compress results folder
!zip -r results_backup.zip results/
print("\n✅ Results compressed!")
print("📦 File: results_backup.zip")

# Download to local machine
from google.colab import files
files.download('results_backup.zip')

---

## Important Notes:

### 1. Enable GPU
- Go to: **Runtime → Change runtime type → Hardware accelerator: GPU**
- Recommended: T4 or better

### 2. Session Timeout
- **Free Colab:** ~12 hours per session
- **Colab Pro:** ~24 hours
- **Solution:** Use batch execution with `--start` and `--end` flags

### 3. If Session Disconnects
1. Re-mount Drive (Cell #2)
2. Change to project directory (Cell #3)
3. Continue from where you left off using `--start X`

### 4. Storage
- Results are saved in your Google Drive
- 180 experiments ≈ 200-300 MB

### 5. Common Issues

**"CUDA out of memory":**
```python
# Already set to batch_size=1, should not occur
# If it does, restart runtime and try again
```

**"Module not found":**
```python
!pip install -r requirements.txt
```

**Path errors:**
```python
import os
print(os.getcwd())
print(os.listdir('.'))
```

### 6. Time Estimates

| Dataset | Model | Runs | Est. Time |
|---------|-------|------|----------|
| MNIST | logistic | 60 | ~1 hour |
| FashionMNIST | logistic | 60 | ~1 hour |
| CIFAR-10 | simple_cnn | 60 | ~2-3 hours |
| **Total** | | **180** | **~4-5 hours** |

---

**Tip:** Always keep your results backed up in Google Drive! 🎯